In [ ]:
import os, sys
sys.path.append("..")

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 

import numpy as np
import torch
import torch.nn as nn
import torchvision
import gc

from src import distributions
import torch.nn.functional as F

from src.resnet2 import ResNet_D
from src.cunet import CUNet

from src.tools import unfreeze, freeze
from src.tools import load_dataset, get_Z_pushed_loader_stats
from src.fid_score import calculate_frechet_distance
from src.tools import weights_init_D
from src.plotters import plot_random_Z_images, plot_Z_images

from copy import deepcopy
import json

from tqdm import tqdm_notebook as tqdm
from IPython.display import clear_output

import wandb
from src.tools import fig2data, fig2img # for wandb

# This needed to use dataloaders for some datasets
from PIL import PngImagePlugin
LARGE_ENOUGH_NUMBER = 100
PngImagePlugin.MAX_TEXT_CHUNK = LARGE_ENOUGH_NUMBER * (1024**2)

## Main Config

In [ ]:
DEVICE_IDS = [0]

# DATASET2, DATASET2_PATH = 'handbag', '../../data/handbag_128.hdf5'
DATASET2, DATASET2_PATH = 'shoes', '../../data/shoes_128.hdf5'
DATASET1, DATASET1_PATH = 'dtd', '../../data/dtd/images'

# DATASET1, DATASET1_PATH = 'handbag', '../../data/handbag_128.hdf5'
# DATASET2, DATASET2_PATH = 'shoes', '../../data/shoes_128.hdf5'

# DATASET1, DATASET1_PATH = 'outdoor', '../../data/outdoor_128.hdf5'
# DATASET2, DATASET2_PATH = 'church', '../../data/church_128.hdf5'

# DATASET2, DATASET2_PATH = 'celeba_female', '../../data/img_align_celeba'
# DATASET1, DATASET1_PATH = 'aligned_anime_faces', '../../data/aligned_anime_faces'

T_ITERS = 10
f_LR, T_LR = 1e-4, 1e-4
IMG_SIZE = 64

ZC = 128
Z_STD = 1.

BATCH_SIZE = 64
Z_SIZE = 4

PLOT_INTERVAL = 500
COST = 'energy' # 'laplacian' #'gaussian' #'weak_mse' # 
CPKT_INTERVAL = 2000
MAX_STEPS = 100001
SEED = 0xBADBEEF

GAMMA0, GAMMA1 = 0.0, 0.333
GAMMA_ITERS = 20000

# CONTINUE = -1 <-- to continue training from a XX9999 checkpoint

EXP_NAME = f'{DATASET1}_{DATASET2}_T{T_ITERS}_{COST}_{IMG_SIZE}'
OUTPUT_PATH = '../checkpoints/{}/{}_{}_{}/'.format(COST, DATASET1, DATASET2, IMG_SIZE)

## Preparation

In [ ]:
config = dict(
    DATASET1=DATASET1,
    DATASET2=DATASET2, 
    T_ITERS=T_ITERS,
    f_LR=f_LR, T_LR=T_LR,
    BATCH_SIZE=BATCH_SIZE
)

AUGMENTED_DATASETS = ['dtd']
FID_EPOCHS = 50 if DATASET1 in AUGMENTED_DATASETS else 1
    
assert torch.cuda.is_available()
torch.cuda.set_device(f'cuda:{DEVICE_IDS[0]}')
torch.manual_seed(SEED); np.random.seed(SEED)

if not os.path.exists(OUTPUT_PATH):
    os.makedirs(OUTPUT_PATH)

## Loading stats

In [ ]:
filename = '../stats/{}_{}_test.json'.format(DATASET2, IMG_SIZE)
with open(filename, 'r') as fp:
    data_stats = json.load(fp)
    mu_data, sigma_data = data_stats['mu'], data_stats['sigma']
del data_stats

## Prepare Samplers (X, Y)

In [ ]:
X_sampler, X_test_sampler = load_dataset(DATASET1, DATASET1_PATH, img_size=IMG_SIZE)
Y_sampler, Y_test_sampler = load_dataset(DATASET2, DATASET2_PATH, img_size=IMG_SIZE)
    
torch.cuda.empty_cache(); gc.collect()
clear_output()

# Initializing Networks

In [ ]:
f = ResNet_D(IMG_SIZE, nc=3).cuda()
f.apply(weights_init_D)
T = CUNet(3, 3, ZC, base_factor=48).cuda()

if len(DEVICE_IDS) > 1:
    T = nn.DataParallel(T, device_ids=DEVICE_IDS)
    f = nn.DataParallel(f, device_ids=DEVICE_IDS)
    
print('T params:', np.sum([np.prod(p.shape) for p in T.parameters()]))
print('f params:', np.sum([np.prod(p.shape) for p in f.parameters()]))

In [ ]:
T_opt = torch.optim.Adam(T.parameters(), lr=T_LR, weight_decay=1e-10)
f_opt = torch.optim.Adam(f.parameters(), lr=f_LR, weight_decay=1e-10)
T_scheduler = torch.optim.lr_scheduler.MultiStepLR(T_opt, milestones=[15000, 25000, 40000, 55000, 70000], gamma=0.5)
f_scheduler = torch.optim.lr_scheduler.MultiStepLR(f_opt, milestones=[15000, 25000, 40000, 55000, 70000], gamma=0.5)

# if CONTINUE > -1:
#     T_opt.load_state_dict(torch.load(os.path.join(OUTPUT_PATH, f'T_opt_{SEED}_{CONTINUE}.pt')))
#     T_scheduler.load_state_dict(torch.load(os.path.join(OUTPUT_PATH, f'T_scheduler_{SEED}_{CONTINUE}.pt')))
#     T.load_state_dict(torch.load(os.path.join(OUTPUT_PATH, f'T_{SEED}_{CONTINUE}.pt')))
#     f_opt.load_state_dict(torch.load(os.path.join(OUTPUT_PATH, f'f_opt_{SEED}_{CONTINUE}.pt')))
#     f.load_state_dict(torch.load(os.path.join(OUTPUT_PATH, f'f_{SEED}_{CONTINUE}.pt')))
#     f_scheduler.load_state_dict(torch.load(os.path.join(OUTPUT_PATH, f'f_scheduler_{SEED}_{CONTINUE}.pt')))

In [ ]:
torch.manual_seed(0xBADBEEF); np.random.seed(0xBADBEEF)
X_fixed, Y_fixed = X_sampler.sample(10), Y_sampler.sample(10)
X_fixed = X_fixed[:,None].repeat(1,4,1,1,1)
with torch.no_grad():
    Z_fixed = torch.randn(10, 4, ZC, 1, 1, device='cuda') * Z_STD
    XZ_fixed = (X_fixed, Z_fixed)
del X_fixed

X_test_fixed, Y_test_fixed = X_test_sampler.sample(10), Y_test_sampler.sample(10)
X_test_fixed = X_test_fixed[:,None].repeat(1,4,1,1,1)
with torch.no_grad():
    Z_test_fixed = torch.randn(10, 4, ZC, 1, 1, device='cuda') * Z_STD
    XZ_test_fixed = (X_test_fixed, Z_test_fixed,)
del X_test_fixed

### Plots Test

In [ ]:
fig, axes = plot_Z_images(XZ_fixed, Y_fixed, T)
fig, axes = plot_random_Z_images(X_sampler, ZC, Z_STD, Y_sampler, T)
fig, axes = plot_Z_images(XZ_test_fixed, Y_test_fixed, T)
fig, axes = plot_random_Z_images(X_test_sampler, ZC, Z_STD, Y_test_sampler, T)

# Run Training

In [ ]:
wandb.init(name=EXP_NAME, project='unpairedot', entity='gunsandroses', config=config)
pass

In [ ]:
for step in tqdm(range(CONTINUE+1, MAX_STEPS)):
    gamma = min(GAMMA1, GAMMA0 + (GAMMA1-GAMMA0) * step / GAMMA_ITERS)
    # T optimization
    unfreeze(T); freeze(f)
    for t_iter in range(T_ITERS): 
        T_opt.zero_grad()
        X = X_sampler.sample(BATCH_SIZE)[:,None].repeat(1,Z_SIZE,1,1,1)
        with torch.no_grad():
            Z = torch.randn(BATCH_SIZE, Z_SIZE, ZC, 1, 1, device='cuda') * Z_STD
        T_XZ = T(
            X.flatten(start_dim=0, end_dim=1), Z.flatten(start_dim=0, end_dim=1)
        ).permute(1,2,3,0).reshape(3, IMG_SIZE, IMG_SIZE, -1, Z_SIZE).permute(3,4,0,1,2)
        
        if COST == 'weak_mse':
            # Weak quadratic cost (normalized by DIM)
            T_var = T_XZ.var(dim=1).mean()
            T_loss = F.mse_loss(X[:,0], T_XZ.mean(dim=1)).mean() - \
            f(T_XZ.flatten(start_dim=0, end_dim=1)).mean() +  T_var * (1 - gamma - 1. / Z_SIZE)
        elif COST == 'energy':
            # Energy-based quadratic cost (for distance-induced kernel)
            T_var = .5 * torch.cdist(T_XZ.flatten(start_dim=2), T_XZ.flatten(start_dim=2)).mean() * Z_SIZE / (Z_SIZE -1)
            T_loss = (X-T_XZ).flatten(start_dim=2).norm(dim=2).mean() - \
            f(T_XZ.flatten(start_dim=0, end_dim=1)).mean() - gamma * T_var
        elif COST == 'gaussian':
            # Gaussian kernel (normalized by DIM)
            idx = torch.triu_indices(Z_SIZE, Z_SIZE, offset=1)
            T_var = 1 - torch.exp(
                -.5*(T_XZ[:,idx[0]]-T_XZ[:,idx[1]]).square().flatten(start_dim=2).mean(dim=2)
            ).mean()
            T_loss = 1 - torch.exp(-0.5 * (X-T_XZ).square().flatten(start_dim=2).mean(dim=2)).mean() -\
            f(T_XZ.flatten(start_dim=0, end_dim=1)).mean() - gamma * T_var
        elif COST == 'laplacian':
            # Laplacian kernel (normalized by DIM)
            idx = torch.triu_indices(Z_SIZE, Z_SIZE, offset=1)
            T_var = 1 - torch.exp(
                -(T_XZ[:,idx[0]]-T_XZ[:,idx[1]]).square().flatten(start_dim=2).mean(dim=2).sqrt()
            ).mean()
            T_loss = 1 - torch.exp(-(X-T_XZ).square().flatten(start_dim=2).mean(dim=2).sqrt()).mean() -\
            f(T_XZ.flatten(start_dim=0, end_dim=1)).mean() - gamma * T_var
        else:
            raise Exception("I do know know this cost!")
        
        T_loss.backward(); T_opt.step()
    T_scheduler.step()
    wandb.log({'var' : T_var.item()}, step=step)
    del T_loss, T_XZ, X, Z, T_var; gc.collect(); torch.cuda.empty_cache()

    # f optimization
    freeze(T); unfreeze(f)
    X = X_sampler.sample(BATCH_SIZE)
    with torch.no_grad():
        Z = torch.randn(BATCH_SIZE, ZC, 1, 1, device='cuda') * Z_STD
        T_XZ = T(X,Z)
    Y = Y_sampler.sample(BATCH_SIZE)
    f_opt.zero_grad()
    f_loss = f(T_XZ).mean() - f(Y).mean()
    f_loss.backward(); f_opt.step(); f_scheduler.step()
    wandb.log({f'f_loss' : f_loss.item()}, step=step)
    del f_loss, Y, X, T_XZ, Z; gc.collect(); torch.cuda.empty_cache()
            
    if step % PLOT_INTERVAL == 0:
        print('Plotting')
        clear_output(wait=True)
        
        fig, axes = plot_Z_images(XZ_fixed, Y_fixed, T)
        wandb.log({'Fixed Images' : [wandb.Image(fig2img(fig))]}, step=step) 
        plt.show(fig); plt.close(fig) 
        
        fig, axes = plot_random_Z_images(X_sampler, ZC, Z_STD,  Y_sampler, T)
        wandb.log({'Random Images' : [wandb.Image(fig2img(fig))]}, step=step) 
        plt.show(fig); plt.close(fig) 
        
        fig, axes = plot_Z_images(XZ_test_fixed, Y_test_fixed, T)
        wandb.log({'Fixed Test Images' : [wandb.Image(fig2img(fig))]}, step=step) 
        plt.show(fig); plt.close(fig) 
        
        fig, axes = plot_random_Z_images(X_test_sampler, ZC, Z_STD,  Y_test_sampler, T)
        wandb.log({'Random Test Images' : [wandb.Image(fig2img(fig))]}, step=step) 
    
    if step % CPKT_INTERVAL == CPKT_INTERVAL - 1:
        freeze(T); 
        torch.save(T.state_dict(), os.path.join(OUTPUT_PATH, f'T_{SEED}_{step}.pt'))
#         torch.save(f.state_dict(), os.path.join(OUTPUT_PATH, f'f_{SEED}_{step}.pt'))
#         torch.save(f_opt.state_dict(), os.path.join(OUTPUT_PATH, f'f_opt_{SEED}_{step}.pt'))
#         torch.save(T_opt.state_dict(), os.path.join(OUTPUT_PATH, f'T_opt_{SEED}_{step}.pt'))
#         torch.save(f_scheduler.state_dict(), os.path.join(OUTPUT_PATH, f'f_scheduler_{SEED}_{step}.pt'))
#         torch.save(T_scheduler.state_dict(), os.path.join(OUTPUT_PATH, f'T_scheduler_{SEED}_{step}.pt'))
        
        print('Computing FID')
        mu, sigma = get_Z_pushed_loader_stats(T, X_test_sampler.loader, ZC=ZC, Z_STD=Z_STD, n_epochs=FID_EPOCHS)
        fid = calculate_frechet_distance(mu_data, sigma_data, mu, sigma)
        wandb.log({f'FID (Test)' : fid}, step=step)
        del mu, sigma
    
    gc.collect(); torch.cuda.empty_cache()